___
# Analyze Inferred-GRN Robustness
___
- Simulate random node failures and hub-targeted removals
- Batch-process phase-specific network directories
- Outputs: robustness plots per network and an aggregate AUC summary

We evaluate directed GRNs across `data/inferred_grn/phase1` through `phase4`, comparing random node failures to degree-based targeted attacks while tracking the giant connected component (GCC) fraction.  
- Edge-list format is detected automatically (tab, comma, semicolon, pipe, or whitespace).  
- GCC is measured on the **weakly connected** component by default (configurable).  

In [1]:
# import modules
from __future__ import annotations

import random
from pathlib import Path
from typing import Iterable, List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
## infers the delimiter used in an edge list file
def guess_delimiter(path: str | Path, sample_lines: int = 20) -> Optional[str]:
    candidates = ["	", ",", ";", "|"]
    with open(path, "r", encoding="utf-8", errors="ignore") as handle:
        for _ in range(sample_lines):
            line = handle.readline()
            if not line:
                break
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            for delim in candidates:
                if delim in line:
                    return delim
    return None

# loads an edge list into a NetworkX graph
def load_graph_from_edgelist(
    path: str | Path,
    *,
    directed: bool = True,
    weighted: bool = False,
) -> nx.Graph:
    path = Path(path)
    delim = guess_delimiter(path)
    create_using = nx.DiGraph() if directed else nx.Graph()
    read_kwargs = {
        "create_using": create_using,
        "nodetype": str,
        "comments": "#",
    }
    if delim is not None:
        read_kwargs["delimiter"] = delim
    if weighted:
        read_kwargs["data"] = [("weight", float)]
    else:
        read_kwargs["data"] = False
    G = nx.read_edgelist(path, **read_kwargs)
    G.remove_edges_from(nx.selfloop_edges(G))
    if G.number_of_nodes() == 0:
        raise ValueError(f"No nodes loaded from {path}. Check delimiter/format.")
    return G

# iterates graph components according to the selected mode
def iter_components(G: nx.Graph, component_mode: str = "weak"):
    if component_mode == "strong":
        if not G.is_directed():
            raise ValueError("strong mode only support directed graph")
        yield from nx.strongly_connected_components(G)
    elif component_mode == "weak":
        if G.is_directed():
            yield from nx.weakly_connected_components(G)
        else:
            yield from nx.connected_components(G)
    elif component_mode == "undirected":
        if G.is_directed():
            yield from nx.connected_components(G.to_undirected())
        else:
            yield from nx.connected_components(G)
    else:
        raise ValueError(f"Unknown component_mode: {component_mode}")

# The function extract_reference_component extracts the largest component
def extract_reference_component(G: nx.Graph, component_mode: str = "weak") -> nx.Graph:
    # select the largest component in component mode
    if G.number_of_nodes() == 0:
        return G
    comps = list(iter_components(G, component_mode=component_mode))
    if len(comps) <= 1:
        return G
    largest = max(comps, key=len)
    return G.subgraph(largest).copy()

# computes the GCC fraction of the graph
def gcc_fraction(G: nx.Graph, component_mode: str = "weak") -> float:
    if G.number_of_nodes() == 0:
        return 0.0
    largest = max((len(c) for c in iter_components(G, component_mode=component_mode)), default=0)
    return largest / max(1, G.number_of_nodes())

In [3]:
# creates the degree-based attack order
def targeted_attack_order_degree(G: nx.Graph, mode: str = "total") -> List[str]:
    if mode not in {"total", "in", "out"}:
        raise ValueError("chose 'total' or 'in' or 'out'")
    if mode in {"in", "out"} and not G.is_directed():
        raise ValueError("undirtected graph can't use in/out mode")

    H = G.copy()
    order: List[str] = []
    while H.number_of_nodes() > 0:
        if mode == "total":
            deg_pairs = list(H.degree())
        elif mode == "in":
            deg_pairs = list(H.in_degree())
        else:  # mode == "out"
            deg_pairs = list(H.out_degree())
        max_deg = max((d for _, d in deg_pairs), default=0)
        candidates = [n for n, d in deg_pairs if d == max_deg]
        node = sorted(candidates)[0]
        order.append(node)
        H.remove_node(node)
    return order

# generates multiple random removal orders
def random_failure_orders(G: nx.Graph, n_trials: int, seed: int = 42) -> List[List[str]]:
    rng = random.Random(seed)
    nodes = list(G.nodes())
    return [rng.sample(nodes, len(nodes)) for _ in range(n_trials)]

# simulates GCC curves during node removal
def simulate_removal_curve(
    G: nx.Graph,
    removal_order: List[str],
    *,
    component_mode: str = "weak",
) -> Dict[str, np.ndarray]:
    H = G.copy()
    N = H.number_of_nodes()
    removed_frac = np.zeros(N + 1)
    gcc_frac = np.zeros(N + 1)
    removed_frac[0] = 0.0
    gcc_frac[0] = gcc_fraction(H, component_mode=component_mode)

    for i, node in enumerate(removal_order, start=1):
        if H.has_node(node):
            H.remove_node(node)
        removed_frac[i] = i / N
        gcc_frac[i] = gcc_fraction(H, component_mode=component_mode)
    return {"removed_frac": removed_frac, "gcc_frac": gcc_frac}

# summarizes random failure curves
def aggregate_random_failures(
    G: nx.Graph,
    *,
    n_trials: int = 20,
    seed: int = 42,
    component_mode: str = "weak",
) -> Dict[str, np.ndarray]:
    orders = random_failure_orders(G, n_trials=n_trials, seed=seed)
    curves = [simulate_removal_curve(G, o, component_mode=component_mode)["gcc_frac"] for o in orders]
    arr = np.vstack(curves)
    mean_curve = arr.mean(axis=0)
    std_curve = arr.std(axis=0)
    removed_frac = simulate_removal_curve(G, orders[0], component_mode=component_mode)["removed_frac"]
    return {
        "removed_frac": removed_frac,
        "mean_gcc": mean_curve,
        "std_gcc": std_curve,
    }

# plots and saves the robustness curves
def plot_robustness(
    name: str,
    phase: str,
    removed_frac: np.ndarray,
    random_mean: np.ndarray,
    random_std: np.ndarray,
    targeted_curve: np.ndarray,
    *,
    component_mode: str,
    target_mode: str,
    base_dir: str | Path = "./",
    show: bool = True,
) -> Path:
    base_dir = Path(base_dir)
    fig_dir = base_dir / phase
    fig_dir.mkdir(parents=True, exist_ok=True)
    fig, ax = plt.subplots(figsize=(6.0, 4.5))
    ax.plot(removed_frac, random_mean, label="Random failure (mean)")
    ax.fill_between(removed_frac, random_mean - random_std, random_mean + random_std, alpha=0.2, label="Random ±1σ")
    ax.plot(removed_frac, targeted_curve, label=f"Targeted attack ({target_mode})")
    ax.set_xlabel("Removed fraction of nodes")
    ax.set_ylabel("GCC fraction (|C_max| / N)")
    ax.set_title(f"Robustness: {phase} / {name}")
    ax.legend()
    ax.grid(True, linestyle="--", alpha=0.4)
    fig_path = fig_dir / f"{name}.png"
    fig.tight_layout()
    fig.savefig(fig_path, dpi=200)
    if show:
        plt.show()
    else:
        plt.close(fig)
    return fig_path


In [4]:
# infers the phase name from the file path
def infer_phase(path: Path, default: str = "unknown") -> str:
    for parent in path.parents:
        name = parent.name
        if name.lower().startswith("phase"):
            return name
    return default

# evaluates robustness for a single network
def analyze_one_file(
    path: str | Path,
    *,
    name: Optional[str] = None,
    directed: bool = True,
    weighted: bool = False,
    component_mode: str = "weak",
    target_mode: str = "total",
    n_random_trials: int = 20,
    seed: int = 42,
    fig_base: str | Path = "./",
    show_plot: bool = True,
) -> Dict[str, object]:
    path = Path(path)
    if name is None:
        name = path.stem

    phase = infer_phase(path)

    print(f"[INFO] Loading: {path}")
    G = load_graph_from_edgelist(path, directed=directed, weighted=weighted)
    raw_nodes, raw_edges = G.number_of_nodes(), G.number_of_edges()

    G_ref = extract_reference_component(G, component_mode=component_mode)
    ref_nodes, ref_edges = G_ref.number_of_nodes(), G_ref.number_of_edges()
    if ref_nodes != raw_nodes:
        print(f"[INFO] Using largest {component_mode} component: {ref_nodes}/{raw_nodes} nodes")

    rand_stats = aggregate_random_failures(
        G_ref,
        n_trials=n_random_trials,
        seed=seed,
        component_mode=component_mode,
    )
    order_tgt = targeted_attack_order_degree(G_ref, mode=target_mode)
    tgt_curve = simulate_removal_curve(G_ref, order_tgt, component_mode=component_mode)["gcc_frac"]

    random_auc = float(np.trapz(rand_stats["mean_gcc"], rand_stats["removed_frac"]))
    targeted_auc = float(np.trapz(tgt_curve, rand_stats["removed_frac"]))
    print(f"[INFO] Random AUC   : {random_auc:.6f}")
    print(f"[INFO] Targeted AUC : {targeted_auc:.6f}")

    fig_path = plot_robustness(
        name,
        phase,
        rand_stats["removed_frac"],
        rand_stats["mean_gcc"],
        rand_stats["std_gcc"],
        tgt_curve,
        component_mode=component_mode,
        target_mode=target_mode,
        base_dir=fig_base,
        show=show_plot,
    )

    print(f"[OK] Saved figure: {fig_path}")

    return {
        "phase": phase,
        "network": name,
        "source_path": str(path),
        "raw_nodes": raw_nodes,
        "raw_edges": raw_edges,
        "ref_nodes": ref_nodes,
        "ref_edges": ref_edges,
        "component_mode": component_mode,
        "target_mode": target_mode,
        "n_random_trials": n_random_trials,
        "seed": seed,
        "initial_gcc": rand_stats["mean_gcc"][0],
        "random_auc": random_auc,
        "targeted_auc": targeted_auc,
        "figure_path": str(fig_path),
    }


In [5]:
# runs robustness analysis over multiple networks
def analyze_directory(
    base_dirs: Iterable[str | Path],
    *,
    patterns: str = "*.ncol,*.txt,*.tsv,*.csv",
    directed: bool = True,
    weighted: bool = False,
    component_mode: str = "weak",
    target_mode: str = "total",
    n_random_trials: int = 20,
    seed: int = 42,
    fig_base: str | Path = "./",
    summary_dir: str | Path = "./",
) -> pd.DataFrame:
    if isinstance(base_dirs, (str, Path)):
        base_dirs = [base_dirs]

    all_paths: List[Path] = []
    for base in base_dirs:
        base = Path(base)
        for pattern in patterns.split(","):
            pattern = pattern.strip()
            if not pattern:
                continue
            all_paths.extend(base.glob(f"**/{pattern}"))

    unique_paths = sorted({p.resolve() for p in all_paths})
    if not unique_paths:
        print(f"[WARN] No files matched in {base_dirs} with patterns '{patterns}'")
        return pd.DataFrame(columns=["phase", "network", "random_auc", "targeted_auc"])

    records: List[Dict[str, object]] = []
    total = len(unique_paths)
    for idx, path in enumerate(unique_paths, start=1):
        if idx > 1:
            print()
        print(f"[BATCH {idx}/{total}] {path}")
        try:
            result = analyze_one_file(
                path,
                name=path.stem,
                directed=directed,
                weighted=weighted,
                component_mode=component_mode,
                target_mode=target_mode,
                n_random_trials=n_random_trials,
                seed=seed,
                fig_base=fig_base,
                show_plot=False,
            )
            records.append(result)
        except Exception as exc:
            print(f"[ERROR] {path}: {exc}")
            records.append({
                "phase": infer_phase(path),
                "network": path.stem,
                "source_path": str(path),
                "error": str(exc),
            })

    full_df = pd.DataFrame(records)

    summary_df = full_df.copy()
    if "error" in summary_df.columns:
        summary_df = summary_df[summary_df["error"].isna()]
    summary_df = summary_df.dropna(subset=["random_auc", "targeted_auc"], how="any")
    summary_df = summary_df.loc[:, ["phase", "network", "random_auc", "targeted_auc"]]

    summary_dir = Path(summary_dir)
    summary_dir.mkdir(parents=True, exist_ok=True)
    summary_path = summary_dir / "robustness_summary.csv"
    summary_df.to_csv(summary_path, index=False)
    print(f"[OK] Saved summary table: {summary_path}")

    return summary_df.reset_index(drop=True)


In [6]:
root_dir = Path("../../data/inferred_grn")
phase_dirs = [root_dir / f"phase{i}" for i in range(1, 5)]

Robustness_results = analyze_directory(
    phase_dirs,
    patterns="*.ncol",
    directed=True,
    weighted=False,
    component_mode="weak",
    target_mode="total",
    n_random_trials=20,
    seed=42,
    fig_base="../../data/result/robustness/figure",
    summary_dir="../../data/result/robustness",
)

[BATCH 1/40] /home/Data_Drive_8TB_2/Controllability/single-cell-grn-control/data/inferred_grn/phase1/Blastomeres.ncol
[INFO] Loading: /home/Data_Drive_8TB_2/Controllability/single-cell-grn-control/data/inferred_grn/phase1/Blastomeres.ncol


/tmp/ipykernel_1672719/1604728258.py:47: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  random_auc = float(np.trapz(rand_stats["mean_gcc"], rand_stats["removed_frac"]))
/tmp/ipykernel_1672719/1604728258.py:48: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  targeted_auc = float(np.trapz(tgt_curve, rand_stats["removed_frac"]))


[INFO] Random AUC   : 0.589191
[INFO] Targeted AUC : 0.032857
[OK] Saved figure: ../../data/result/robustness/figure/phase1/Blastomeres.png

[BATCH 2/40] /home/Data_Drive_8TB_2/Controllability/single-cell-grn-control/data/inferred_grn/phase1/Enveloping_Layer.ncol
[INFO] Loading: /home/Data_Drive_8TB_2/Controllability/single-cell-grn-control/data/inferred_grn/phase1/Enveloping_Layer.ncol
[INFO] Random AUC   : 0.744938
[INFO] Targeted AUC : 0.062697
[OK] Saved figure: ../../data/result/robustness/figure/phase1/Enveloping_Layer.png

[BATCH 3/40] /home/Data_Drive_8TB_2/Controllability/single-cell-grn-control/data/inferred_grn/phase1/Primordial_Germ_cells.ncol
[INFO] Loading: /home/Data_Drive_8TB_2/Controllability/single-cell-grn-control/data/inferred_grn/phase1/Primordial_Germ_cells.ncol
[INFO] Using largest weak component: 257/259 nodes
[INFO] Random AUC   : 0.747251
[INFO] Targeted AUC : 0.091122
[OK] Saved figure: ../../data/result/robustness/figure/phase1/Primordial_Germ_cells.png

[BA

In [7]:
Robustness_results

,phase,network,random_auc,targeted_auc
0,phase1,Blastomeres,0.589191,0.032857
1,phase1,Enveloping_Layer,0.744938,0.062697
2,phase1,Primordial_Germ_cells,0.747251,0.091122
3,phase2,Ectoderm,0.632711,0.036512
4,phase2,Enveloping_Layer,0.665720,0.041767
5,phase2,Other_Axial_Mesoderm,0.652983,0.036349
6,phase2,Other_Mesendoderm,0.615644,0.028812
7,phase2,Primordial_Germ_cells,0.629895,0.044960
8,phase3,Ectoderm,0.633301,0.037008
9,phase3,Enveloping_Layer,0.605942,0.031439
